## *Instalaciones*

In [38]:
!pip install openai
!pip install PyWavelets # Para realizar transformadas wavelet, útiles en eliminación de ruido y análisis de señales

## *Configuraciones principales y librerias de agentes*

In [39]:
from google.colab import drive
drive.mount('/content/drive')

path_start = "/content/drive/My Drive/Colab Notebooks/Iniciativas/20240429_Unir/Tesis/0_utility"

%cd "{path_start}"

import sett
import classification_model_tools
import sentimental_model_tools

path_app = sett.path_app
path_model = sett.path_model

%cd "{path_app}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Iniciativas/20240429_Unir/Tesis/0_utility
/content/drive/My Drive/Colab Notebooks/Iniciativas/20240429_Unir/Tesis


## *Importacion de librerias y configuraciones de modelos*

In [40]:
import os
import openai
from datetime import datetime
import json

os.environ['OPENAI_API_KEY'] = sett.apikey_openai
openai.api_key = sett.apikey_openai

## *Prueba de Tools de Librerias*



In [41]:
fecha_de_dato_predecir = '2025-03-01'
paridad = "EUR/USD"

json_analisis_tecnico = classification_model_tools.response_analisis_tecnico(fecha_de_dato_predecir, paridad)

print(str(json_analisis_tecnico))

{'fecha_de_analisis': Timestamp('2025-03-01 00:00:00'), 'recomendacion': 'Venta', 'probabilidad': 0.84, 'justificacion': 'La prediccion para el dia siguiente muestra un movimiento negativo con una probabilidad estimada del 0.84. Esta recomendacion se basa en los indicadores tecnicos (RSI, ATR, ROC, WR) y el analisis historico del modelo.'}


## *Clase Multiagente*

In [31]:
class MultiAgentSupervisor:
    """
    Clase que supervisa la ejecución de múltiples agentes especializados en el análisis de datos
    y sentimiento del mercado FOREX para una paridad de divisas específica.

    Args:
        paridad (str): Código de la paridad de divisas (e.g., 'EUR/USD').
        descripcion_paridad (str): Descripción de la paridad de divisas.
        fecha_hoy (str): Fecha actual en formato 'YYYY-MM-DD'.
    """

    def __init__(self, paridad, descripcion_paridad, fecha_hoy):
        """
        Inicializa el supervisor de múltiples agentes con la paridad de divisas y la fecha actual.
        """

        self.llm_orchestrator = self.agente_planificador  # LLM que actúa como orquestador

        self.agents = {
                       "analisis_sentimiento": self.agente_analisis_sentimiento,
                       "clasificar_datos": self.agente_clasificar_datos,
                      }

        self.paridad = paridad
        self.descripcion_paridad = descripcion_paridad
        self.fecha_hoy = fecha_hoy

    def agente_planificador(self, user_prompt):
        """
        Orquesta la ejecución de agentes en función de la consulta del usuario.

        Args:
            user_prompt (str): Entrada del usuario con la consulta sobre el mercado FOREX.

        Returns:
            tuple: Lista de agentes a utilizar y la fecha extraída de la consulta.
        """

        agent_system_prompt = "Actúa como un orquestador de agentes."\
                              f"Eres un agente que orquestada agentes para un sistema de decisión de trading en el mercado FOREX y de la paridad '{self.paridad}' ('{self.descripcion_paridad}'). "\
                              f"Si el usuario menciona la palabra **paridad** sin especificar que monedas en su consulta, asume que se refiere a la paridad '{self.descripcion_paridad}'."

        agent_user_prompt = f"Tu tarea es analizar la siguiente consulta del usuario '{user_prompt}' y determinar el nombre del agente que debe de "\
                            "manejarla. Sigue estas reglas estrictas:\n\n"\
                            f"1. Si la consulta NO esta relacionada a la paridad '{self.descripcion_paridad}' ('{self.paridad}'), ni con datos históricos "\
                            "ni sentimiento de mercado, responde únicamente con 'NADA' (en mayúsculas, sin comillas).\n\n"\
                            "2. Si la consulta menciona sobre **noticias, redes sociales, comentarios u opiniones**, usa el agente 'analisis_sentimiento'.\n"\
                            "3. Si la consulta menciona sobre **datos históricos, tendencias, fluctuaciones o predicciones**, usa el agente 'clasificar_datos'.\n"\
                            "4. Usa **ambos agentes ('analisis_sentimiento', 'clasificar_datos') solo si la pregunta incluye simultáneamente tanto noticias/opiniones como análisis histórico/predicciones**.\n"\
                            "5. Extrae la fecha de la consulta:\n"\
                            "   - Si la consulta menciona una fecha explícita, usa esa fecha.\n"\
                            f"   - Si no hay una fecha en la consulta, asume la fecha de hoy: {self.fecha_hoy}.\n"\
                            "Responde solo con los nombres de los agentes separados por coma, seguidos de la fecha extraída. Si la consulta no aplica, responde solo con 'NADA'.\n\n"\
                            "**Este es un ejemplo de formato de respuesta:**\n"\
                            "- Si se selecciona un solo agente: 'nombre_agente, fecha_extraída'\n"\
                            "- Si se seleccionan ambos agentes: 'analisis_sentimiento, clasificar_datos, fecha_extraída'\n"\
                            "- Si la consulta no aplica: 'NADA'\n\n"

        #print(prompt_intruccion)
        #print("###########################################")

        response = openai.chat.completions.create(
                                                  model = sett.orchestrating_agent_model,
                                                  messages = [
                                                              {"role": "system",
                                                              "content": agent_system_prompt
                                                              },
                                                              {"role": "user",
                                                              "content": agent_user_prompt
                                                              }
                                                            ],
                                                  max_tokens = 70,
                                                  temperature=0.3,
                                                )

        response_agente_principal = response.choices[0].message.content.strip().split(",")

        #print("agente_planificador, response_agente_principal: " + str(response_agente_principal))

        agents_to_use = [item.strip() for item in response_agente_principal[:-1]]

        #print("agente_planificador, agents_to_use: " + str(agents_to_use))

        fecha_dada = response_agente_principal[-1].strip()

        #print("agente_planificador, fecha_dada: " + str(fecha_dada))

        return agents_to_use, fecha_dada

    def agente_analisis_sentimiento(self, fecha_dada):
        """
        Ejecuta el análisis de sentimiento para una fecha específica.

        Args:
            fecha_dada (str): Fecha en formato 'YYYY-MM-DD'.

        Returns:
            str: Interpretación en lenguaje natural del análisis de sentimiento.
        """

        json_response_analisis_sentimiento = sentimental_model_tools.response_analisis_sentimiento(fecha_dada, self.paridad)

        agent_system_prompt = "Eres un agente que interpreta una estructura json que contiene el analiza sentimientos de varios textos que "\
                              f"comentan sobre la paridad '{self.paridad}' del mercado FOREX."

        agent_user_prompt = "Interpreta el siguiente json y conviertelo en un texto que pueda entender una persona que no entienda sobre el "\
                            f"mercado FOREX:\n\njson: {str(json_response_analisis_sentimiento)}"

        response = openai.chat.completions.create(
                                                  model = sett.functional_agent_model,
                                                  messages = [
                                                              { "role": "system",
                                                                "content": agent_system_prompt
                                                              },
                                                              { "role": "user",
                                                                "content": agent_user_prompt
                                                              }
                                                            ],
                                                  #max_tokens=200,
                                                  temperature=0.3
                                                )

        return response.choices[0].message.content.strip()

    def agente_clasificar_datos(self, fecha_dada):
        """
        Ejecuta la clasificación de datos históricos de la paridad.

        Args:
            fecha_dada (str): Fecha en formato 'YYYY-MM-DD'.

        Returns:
            str: Interpretación en lenguaje natural de la clasificación de datos.
        """

        json_response_analisis_tecnico = classification_model_tools.response_analisis_tecnico(fecha_dada, self.paridad)

        agent_system_prompt = "Eres un agente que interpreta una estructura json que contiene la predicción de datos historico de la paridad "\
                              f"'{self.paridad}' del mercado FOREX."

        agent_user_prompt = "Interpreta el siguiente json y conviertelo en un texto que pueda entender una persona que no entienda sobre el "\
                            f"mercado FOREX:\n\njson: {str(json_response_analisis_tecnico)}"

        response = openai.chat.completions.create(
                                                  model = sett.functional_agent_model,
                                                  messages=[
                                                            { "role": "system",
                                                              "content": agent_system_prompt
                                                            },
                                                            { "role": "user",
                                                              "content": agent_user_prompt
                                                            }
                                                          ],
                                                  #max_tokens=200,
                                                  temperature=0.3,
                                                )

        return response.choices[0].message.content.strip()

    def supervise(self, user_prompt):
        """
        Supervisa la ejecución de los agentes en función de la consulta del usuario.

        Args:
            user_prompt (str): Entrada del usuario con la consulta sobre el mercado FOREX.

        Returns:
            dict | str: Resultados de los agentes en un diccionario o una respuesta en lenguaje natural.
        """

        conversational_agent_system_prompt = f"Tú nombre es '{sett.chatbot_name}', éres un experto en el mercado FOREX, especificamente de la paridad '{self.paridad}' ({self.descripcion_paridad}) "\
                                            f"y puedes conversar sobre ese tema; ten en cuenta que si el usuario solo menciona la palabra **paridad** en su consulta, "\
                                            f"asume que se refiere a la paridad '{self.descripcion_paridad}', si menciona otro tipo de paridad indicar que no estas capacitado para "\
                                            f"responder esa pregunta.\n\nConsidera que la fecha del día de hoy es '{self.fecha_hoy}.\n\n"\
                                            "Responde como un asesor financiero pero sin predecir mercados ni tendencias."

        agents_to_use, fecha_dada = self.llm_orchestrator(user_prompt)

        #print("supervise, self.llm_orchestrator: " + str(agents_to_use) + ", " +  str(fecha_dada))

        results = {}

        if agents_to_use:
            for agent_name in agents_to_use:
                if agent_name in self.agents:
                    results[agent_name] = self.agents[agent_name](fecha_dada)
        else:
            response = openai.chat.completions.create(
                                                      model = sett.conversational_model,
                                                      messages=[
                                                                { "role": "system",
                                                                  "content": conversational_agent_system_prompt
                                                                },
                                                                { "role": "user",
                                                                  "content": user_prompt
                                                                }
                                                              ],
                                                      temperature=0.5,
                                                    )

            return response.choices[0].message.content.strip()

        return results

## *Chat con el Chatbot*

In [37]:
if __name__ == "__main__":
    """
    Punto de entrada del programa. Inicializa la supervisión de agentes y gestiona la interacción con el usuario.

    - Define la paridad de divisas y su descripción.
    - Crea una instancia del supervisor MultiAgentSupervisor.
    - Inicia un bucle de conversación con el usuario hasta que ingrese un comando de salida.
    """

    # Parámetros
    paridad = "EUR/USD"
    descripcion_paridad = "euro dólar"
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")

    # Creamos la clase supervisor de multiagentes
    supervisor = MultiAgentSupervisor(paridad, descripcion_paridad, fecha_hoy)

    # Iniciamos la conversación
    while True:
        user_prompt = input("User: ")

        if user_prompt.lower() in ["salir", "exit", "quit"]:
            break

        result = supervisor.supervise(user_prompt)

        print("\n" + sett.chatbot_name + ": ")

        if isinstance(result, dict):
            for agent, output in result.items():
                print(f"{agent.capitalize()}: {output}")
        else:
            print(result)

        print("\n=================================")

User: Hola

Forexin: 
¡Hola! ¿En qué puedo ayudarte hoy en relación a la paridad EUR/USD?

User: salir
